# Yelp API Restaurant Calls By GPS Coordinates

For each coordinate listed in grid_coordinates.csv, return first 1k restaurants found via yelp api and
extract data regarding:

- name
- address & zip
- coordinates
- rating
- review count
- price level
- category
- yelp id

Write data to restaurant_data_coord.csv

In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import csv

# Import API key
from api_keys import api_key

### Perform API calls

- Dataframe from grid_coordinates.csv
- Build function that calls and writes restaurant data to new csv, given pair of coordinates
- Run list of coordinates through function

In [2]:
# Create dataframe from csv file
coords_df = pd.read_csv("Resources/grid_coordinates.csv", 
                        names=["Lat", "Lng"])
coords_df

,Lat,Lng
0,38.871657,-123.109843
1,38.871657,-123.017222
2,38.871657,-122.924601
3,38.871657,-122.831980
4,38.871657,-122.739359
5,38.871657,-122.646738
6,38.871657,-122.554117
7,38.871657,-122.461496
8,38.871657,-122.368875
9,38.871657,-122.276254


In [3]:
# Function returns up to 1000 restaurant listings for input coordinates
def get_restaurants(lat, lng, api_key):
    
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {"Authorization": "Bearer %s" % api_key}
    restaurant_data = []
    count = 0
    
    # Increases search return limit
    for offset in range(0, 1000, 50):
        
        # Set parameters and pass into API calls, radius 8046 meters = 5 miles
        params = {"term":"restaurants", "sort_by":"rating", "latitude":lat, "longitude":lng, "radius":8046, 
                  "limit":50, "offset":offset}
        req = requests.get(url, params=params, headers=headers)
        
        # Breaks if error occurs with search
        if req.status_code == 400:
            break
        elif req.status_code == 200:
            
            # Convert to json
            response = req.json()
            
            # Log history
            count += 1
            print(f"Now processing query set {count} of max 20")
            
            # Breaks if no further entries in query
            if response["businesses"] == []:
                break
            
            else:
                # Iterate through business results and extract data
                for biz in response["businesses"]:
                    
                    # Logic to replace missing price level data with NaN
                    if "price" not in biz:
                        
                        # Replace both price & missing category data with empty string
                        if biz["categories"] == []:
                            restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], 
                                                    biz["coordinates"]["latitude"], 
                                                    biz["coordinates"]["longitude"], 
                                                    biz["location"]["address1"], 
                                                    biz["location"]["zip_code"], biz["rating"], 
                                                    biz["review_count"], "", "", biz["id"]])
                        else:
                            restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], 
                                                    biz["coordinates"]["latitude"], 
                                                    biz["coordinates"]["longitude"], 
                                                    biz["location"]["address1"], 
                                                    biz["location"]["zip_code"], biz["rating"], 
                                                    biz["review_count"], "", biz["categories"][0]["title"], 
                                                    biz["id"]])

                    # Replace missing category data with empty string
                    elif biz["categories"] == []:
                        restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], biz["price"], "", biz["id"]])

                    else:
                        restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], biz["price"], 
                                                biz["categories"][0]["title"], biz["id"]])

    # Write to csv
    with open('Resources/restaurant_data_coords_ratings.csv', 'a', encoding="utf-8") as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(restaurant_data)
    csvFile.close()
    
    # Returns total count of restaurants in coordinates
    return response["total"]


In [8]:
# Track # of coordinates processed
count = 0

# List to track total restaurants found in coordinate
totals_count = []

print("LOG HISTORY OF API CALLS:")
print("---------------------------")

# Loop thru list of coordinates
lat_coords = [lat for lat in coords_df["Lat"]]
lng_coords = [lng for lng in coords_df["Lng"]]

for index in range(len(lat_coords)):
                   
    #  Call get_restaurants fn and append to total_count list
    totals_count.append(get_restaurants(lat_coords[index], lng_coords[index] , api_key))
    
    # Print log history
    count += 1
    rem = len(lat_coords) - count
    print("-----------------------------------------")
    if count == len(lat_coords):
        print("Full list of coordinates processed!")
    elif count == len(lat_coords) - 1:
        print("Now getting results for final coordinates. Almost there!")
        print("-----------------------------------------")
    else:
        print(f"Data retrieval for coordinates {count} complete")
        print(f"Getting results for next coordinates.. there are {rem} left")
        print("-----------------------------------------")

coords_df["Total # of Restaurants"] = totals_count

LOG HISTORY OF API CALLS:
---------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates 1 complete
Getting results for next coordinates.. there are 531 left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates 2 complete
Getting results for next coordinates.. there are 530 left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates 3 complete
Getting results for next coordinates.. there are 529 left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates 4 complete
Getting results for next coordinates.. there are 528 left
-----------------------------------------
Now processing query set 

Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates 34 complete
Getting results for next coordinates.. there are 498 left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates 35 complete
Getting results for next coordinates.. there are 497 left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates 36 complete
Getting results for next coordinates.. there are 496 left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates 37 complete
Getting results for next coordinates.. there are 495 left
-----------------------------------------
N

Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates 66 complete
Getting results for next coordinates.. there are 466 left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates 67 complete
Getting results for next coordinates.. there are 465 left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates 68 complete
Getting results for next coordinates.. there are 464 left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates 69 complete
Getting results for next coordinates.. there are 463 left
-----------------------------------------
N

Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
Now processing query set 4 of max 20
Now processing query set 5 of max 20
Now processing query set 6 of max 20
-----------------------------------------
Data retrieval for coordinates 92 complete
Getting results for next coordinates.. there are 440 left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates 93 complete
Getting results for next coordinates.. there are 439 left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
Now processing query set 4 of max 20
Now processing query set 5 of max 20
Now processing query set 6 of max 20
Now processing query set 7 of max 20
Now processing query set 8 of max 20
Now processing query set 9 of max 20
N

Now processing query set 16 of max 20
Now processing query set 17 of max 20
Now processing query set 18 of max 20
-----------------------------------------
Data retrieval for coordinates 113 complete
Getting results for next coordinates.. there are 419 left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
Now processing query set 4 of max 20
Now processing query set 5 of max 20
Now processing query set 6 of max 20
Now processing query set 7 of max 20
Now processing query set 8 of max 20
Now processing query set 9 of max 20
Now processing query set 10 of max 20
Now processing query set 11 of max 20
Now processing query set 12 of max 20
Now processing query set 13 of max 20
Now processing query set 14 of max 20
Now processing query set 15 of max 20
Now processing query set 16 of max 20
Now processing query set 17 of max 20
Now processing query set 18 of max 20
Now processing query set 

...

Now processing query set 6 of max 20
Now processing query set 7 of max 20
Now processing query set 8 of max 20
Now processing query set 9 of max 20
Now processing query set 10 of max 20
Now processing query set 11 of max 20
Now processing query set 12 of max 20
Now processing query set 13 of max 20
Now processing query set 14 of max 20
Now processing query set 15 of max 20
Now processing query set 16 of max 20
Now processing query set 17 of max 20
Now processing query set 18 of max 20
Now processing query set 19 of max 20
Now processing query set 20 of max 20
-----------------------------------------
Data retrieval for coordinates 414 complete
Getting results for next coordinates.. there are 118 left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates 415 complete
Getting results for next coordinates.. there are 117 left
------------------------------

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates 436 complete
Getting results for next coordinates.. there are 96 left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates 437 complete
Getting results for next coordinates.. there are 95 left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates 438 complete
Getting results for next coordinates.. there are 94 left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates 439 complete
Getting results for next coordinates.. there are 93 left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates 44

Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates 466 complete
Getting results for next coordinates.. there are 66 left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates 467 complete
Getting results for next coordinates.. there are 65 left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
-----------------------------------------
Data retrieval for coordinates 468 complete
Getting results for next coordinates.. there are 64 left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
Now processing query set 4 of max 20
--------------------------------------

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates 496 complete
Getting results for next coordinates.. there are 36 left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates 497 complete
Getting results for next coordinates.. there are 35 left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates 498 complete
Getting results for next coordinates.. there are 34 left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates 499 complete
Getting results for next coordinates.. there are 33 left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates 50

Now processing query set 7 of max 20
Now processing query set 8 of max 20
Now processing query set 9 of max 20
-----------------------------------------
Data retrieval for coordinates 526 complete
Getting results for next coordinates.. there are 6 left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
Now processing query set 4 of max 20
Now processing query set 5 of max 20
-----------------------------------------
Data retrieval for coordinates 527 complete
Getting results for next coordinates.. there are 5 left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
Now processing query set 4 of max 20
-----------------------------------------
Data retrieval for coordinates 528 complete
Getting results for next coordinates.. there are 4 left
-----------------------------------------
Now 

In [9]:
# Show coords_df with updated total # of restaurants
coords_df.head()

,Lat,Lng,Total # of Restaurants
0,38.871657,-123.109843,0
1,38.871657,-123.017222,23
2,38.871657,-122.924601,0
3,38.871657,-122.831980,0
4,38.871657,-122.739359,9


In [10]:
# Save restaurants counts to csv
coords_df.to_csv("Resources/total_count_coords_ratings.csv")